<a href="https://colab.research.google.com/github/thisisanu/online_deep_learning/blob/main/homework3/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()
!nvidia-smi


Sun Nov  9 04:49:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
import shutil

# Ensure we are in /content
os.chdir("/content")
print("Current directory:", os.getcwd())

# If a folder named homework2 exists, remove it completely
if os.path.exists("online_deep_learning"):
    shutil.rmtree("online_deep_learning")
    print("Old online_deep_learning folder removed.")
else:
    print("No existing online_deep_learning folder found.")

!git clone https://github.com/thisisanu/online_deep_learning.git
%cd online_deep_learning/homework3


Current directory: /content
Old online_deep_learning folder removed.
/content
Cloning into 'online_deep_learning'...
remote: Enumerating objects: 481, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 481 (delta 125), reused 19 (delta 3), pack-reused 295 (from 2)
Receiving objects: 100% (481/481), 3.05 MiB | 23.34 MiB/s, done.
Resolving deltas: 100% (312/312), done.
/content
/content/online_deep_learning/homework3


In [ ]:
#download the datasets by running the following command from the main directory
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/classification_data.zip -o ./classification_data.zip && unzip -qo classification_data.zip
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/drive_data.zip -o ./drive_data.zip && unzip -qo drive_data.zip
!ls


bundle.py		 drive_data	 homework	  requirements.txt
classification_data	 drive_data.zip  Homework3.ipynb
classification_data.zip  grader		 README.md


In [ ]:
!pip install numpy torch torchvision tqdm matplotlib
!python3 -m grader homework -v